In [31]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber, FreeSurferSource # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, MapNode, JoinNode        # pypeline engine
from pandas import DataFrame, read_csv
import statsmodels.formula.api as smf

#set output file type for FSL to NIFTI_GZ
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# Set study variables
study_home = '/Users/SEAlab/Documents/PPM'
hcpproc_dir = study_home + '/Data'
output_dir = study_home + '/fMRIproc/longitudinal'
workflow_dir = study_home + '/Workflows'
cov_file = study_home + '/Misc/covariates.csv'
cov_dataframe = read_csv(cov_file,index_col=None)

# set model specifications
model = smf.glm
model_formula = 'Y ~ age'

proc_cores = 2 # number of cores of processing for the workflows

In [ ]:
## Data handling nodes

# grab data
def pull_data(data_dict,data_label):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from glob import glob
    
    file_list = glob(data_dict[data_label])
    file_list = sorted(file_list)
    
    return(file_list)

selectfiles = Node(Function(input_names=['data_dict','data_label'],
                            output_names=['file_list'],
                            function=pull_data), name='selectfiles')
selectfiles.input.data_dict={'cortical_thickness':hcpproc_dir + '/*/MNINonLinear/fsaverage_LR32k/*.thickness.32k_fs_LR.dscalar.nii',
                             'functional_connectivity':hcpproc_dir + ''}

# sink data
substitutions = []
datasink = Node(DataSink(),name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir
datasink.inputs.substitutions = substitutions

In [40]:
# create index for cifti file
def create_indexing(file_list,n_slices):
    from glob import glob
    import nibabel as nib
    import numpy as np
    import hcp_utils as hcp
    from sklearn.utils import Bunch

    img1 = nib.load(file_list[0])
    Y1 = img1.get_fdata()
    N = Y1.shape[0]
    M = Y1.shape[1]
    indexes=Bunch()

    index_labels = ['slice{0}'.format(a) for a in range(0,n_slices)]
    width = int(M/n_slices)
    start=0
    end=width
    i=0
    while end<M:
        indexes[index_labels[i]]=slice(start,end)
        start+=width
        end+=width
        i+=1
    if end-width<M:
        indexes[index_labels[i]]=slice(start,M)
    indexes_list = [indexes[x] for x in indexes.keys()]
    return(indexes_list)

# pull data from the same slice for each file
def agg_data(file_list,index):
    import nibabel as nib
    import numpy as np
    from os.path import basename
    
    for file in file_list:
        img = nib.load(file)
        temp = img.get_fdata()
        temp = temp[:,index]
        temp_file = basename(file).replace('.nii','.slice.npy')
        np.save(temp_file,temp)
        data_stack.append(temp_file)
        
    return(data_stack)

# run models on data
def run_long_models(model,model_formula,data_stack,covariates):
    
    return(p_slice,beta_slice,t_slice)

# reconstitute the results into a cifti file
def combine_data(p_list,beta_list,t_list,index_list):
    
    return(pval_cifti,beta_cifti,tstat_cifti)


In [41]:
# Node to wrap the function that makes the list of indexes
make_indexes = Node(Function(input_names=['file_list','n_slices'], output_names=['indexes_list'], 
                             function=create_indexing),name='create_indexing')

# Node to wrap the function that pulls the data to model
pull_data = MapNode(Function(input_names=['file_list','index'],output_names=['data_stack'],
                             function=agg_data),name='pull_data', iterfield=['index'])

# Node to wrap the function that runs the longitudinal model
run_models = MapNode(Function(input_names=['model','model_formula','data_stack','covariates'],
                              output_names=['p_slice','beta_slice','t_slice'], function=run_long_models),
                     name='run_models',iterfield=['data_stack'])
run_models.inputs.model=model
run_models.inputs.model_formula=model_formula
run_models.inputs.covariates=cov_dataframe

# Node to recombine the data slices for visualization
recombine_data = JoinNode(Function(input_names=['p_list','beta_list','t_list','index_list'],
                                   output_names=['pval_cifti','beta_cifti','tstat_cifti'],function=combine_data),
                          name='recombine_data', joinfield=['p_list','beta_list','t_list','index_list'],
                          joinsource='make_indexes')

In [47]:
from glob import glob
import nibabel as nib
import numpy as np
import hcp_utils as hcp
from sklearn.utils import Bunch

file_list = glob(hcpproc_dir + '/*/MNINonLinear/fsaverage_LR32k/*.thickness.32k_fs_LR.dscalar.nii')
n_slices = 4

img1 = nib.load(file_list[0])
Y1 = img1.get_fdata()
N = Y1.shape[0]
M = Y1.shape[1]
indexes=Bunch()

index_labels = ['slice{0}'.format(a) for a in range(0,n_slices)]
width = int(M/n_slices)
start=0
end=width
i=0
while end<M:
    indexes[index_labels[i]]=slice(start,end)
    start+=width
    end+=width
    i+=1
if end-width<M:
    indexes[index_labels[i]]=slice(start,M)

indexes_list = [indexes[x] for x in indexes.keys()]
    
    #elif N==M:
#else:
    #raise('Data dimensions NxM do not match a scalar image (N=1) nor connectivity matrix (N=M). Did you link the right data?')
    

In [48]:
import nibabel as nib
import numpy as np
from os.path import basename

#for index in indexes_list:
index=indexes_list[0]
print(index)
data_stack = []

for file in file_list:
    img = nib.load(file)
    temp = img.get_fdata()
    print(temp.shape)
    temp = temp[:,index]
    print(temp2.shape)
    temp_file = basename(file).replace('.nii','.slice.npy')
    np.save(temp_file,temp)
    data_stack.append(temp_file)
print(data_stack)

slice(0, 14853, None)
(1, 59412)
(1, 14853)
(1, 59412)
(1, 14853)
(1, 59412)
(1, 14853)
(1, 59412)
(1, 14853)
(1, 59412)
(1, 14853)
['1196_Preg3.thickness.32k_fs_LR.dscalar.slice.npy', '1196_Preg4.thickness.32k_fs_LR.dscalar.slice.npy', '1196_Preg5.thickness.32k_fs_LR.dscalar.slice.npy', '1196_Preg2.thickness.32k_fs_LR.dscalar.slice.npy', '1196_Preg1.thickness.32k_fs_LR.dscalar.slice.npy']


In [49]:
import numpy as np
from os.path import basename

for d in data_stack:
    t = np.load(d)
    print(t.shape)

(1, 14853)
(1, 14853)
(1, 14853)
(1, 14853)
(1, 14853)
